In [1]:
#pragma cling add_include_path("../../include")
#pragma cling add_include_path("../feltor/inc") // Feltor path
#define THRUST_DEVICE_SYSTEM THRUST_DEVICE_SYSTEM_CPP
#include <iostream>
#include "dg/algorithm.h"
// include json and netcdf?

In file included from input_line_8:2:
In file included from ../feltor/inc/dg/algorithm.h:8:
../feltor/inc/dg/backend/config.h:20:9: warning: NOTE: Fast std::fma(a,b,c) not activated! Using a*b+c instead! [-W#pragma-messages]
#pragma message( "NOTE: Fast std::fma(a,b,c) not activated! Using a*b+c instead!")
        ^
In file included from input_line_8:2:
In file included from ../feltor/inc/dg/algorithm.h:11:
In file included from ../feltor/inc/dg/topology/split_and_join.h:4:
In file included from ../feltor/inc/dg/backend/blas1_dispatch_shared.h:12:
In file included from ../feltor/inc/dg/backend/blas1_serial.h:6:
In file included from ../feltor/inc/dg/backend/exblas/exdot_serial.h:25:
In file included from ../feltor/inc/dg/backend/exblas/accumulate.h:19:
../feltor/inc/dg/backend/exblas/config.h:31:9: warning: WARNING: Instruction set below SSE4.1! Deactivating vectorization!
      [-W#pragma-messages]
#pragma message("WARNING: Instruction set below SSE4.1! Deactivating vectorization!")
 

(sec:pdes)=
# Timesteppers for partial differential equations

We now want to demonstrate how to use Feltor to solve partial differential equations.
We use the simple advection diffusion equation as a model equation
\begin{align}
    \frac{\partial \omega}{\partial t} &= -v\cdot \nabla\omega + D \Delta \omega \\
     -\Delta \phi &= \omega \\
     v_x &:= -\partial_y \phi \\
     v_y &:= \partial_x \phi
\end{align}

## Explicit stepping
As long as the diffusion coefficient is small enough to not influence the CFL condition we can compute everything explicitly. In Feltor we simply need a functor implementing the right hand side like we did in the
last chapter. We here repeat the building blocks that regard the timestepper:

```cpp
template<class Geometry, class Matrix, class Container>
struct Equations
{
    void operator()(double t, const Container& omega, Container& omegaDot)
    {
        // solve Poisson equation
        // implement advection term
        // implement diffusion term
    }
};
// Construct init condition
omega = myproject::initial_conditions(grid, js["init"] );

// Construct Equations
myproject::Equations<dg::x::CartesianGrid2d, dg::x::DMatrix,
    dg::x::DVec> rhs( grid, js);

// The timestepper
dg::Adaptive< dg::ERKStep< dg::x::DVec>> adapt(tableau, omega);

// The timeloop
dg::AdaptiveTimeloop<dg::x::DVec> timeloop( adapt, rhs, 
                    dg::pid_control, dg::l2norm, rtol, atol);
for( unsigned u=1; u<=maxout; u++)
{
   
    timeloop.integrate( time, omega, u*deltaT, omega,
                          u < maxout ? dg::to::at_least : dg::to::exact);
    // ...
}
```

## Explicit advection - implicit diffusion
We want to split the PDE into two parts: $E(\phi, \omega) = -\vec v \cdot \nabla \omega$ and $I(\omega) = D\Delta\omega$ with $\phi = S(\omega) = \Delta^{-1}\omega$.
We intend to use a semi-implicit time integrator:
```cpp
template<class Geometry, class Matrix, class Container>
struct Explicit
{
    Explicit(...){}
    void operator()( double t, const Container& omega,
                    Container& k)
    {
        // Solve Phi = S(omega) with e.g. Multigrid
        // Compute k = E(phi,omega)
    }
    void implicit_part( double t, const Container& omega,
                       Container& k)
    {
        // Compute k = D Delta omega
    }
};
```
```{note}
There are several possibilities to partition the explicit, implicit and solver parts into functors (or lambdas in the main program). Generally, it is a good idea to keep all equation related functionality in one class. Since we cannot overload the `operator()` twice we reverted to a little trick, writing the `implicit_part` method that we later bind to the `operator()` of the Implicit class below.
```
```cpp
template<class Geometry, class Matrix, class Container>
struct Implicit
{
    Implicit( Explicit<...>& exp, ...): m_exp(exp) {...}
    void operator()( double t, const Container& omega, Container& k)
    {
        m_exp->implicit_part( t, omega, k);      
    }
```
```{note}
For the solve method we here chose a PCG solver since the Laplace is self-adjoint. Note, how we used  a small lambda wrapper to compute the implicit left hand side. 

Typically, the solver would also write some information about its performance to `std::cout` so that a user is kept informed about the status of the integration.
```
```cpp
    void operator()( double alpha, double t, Container& omega, const Container& rhs)
    {
        auto wrapper = [=]( const auto& x, auto& y){
            // x - a I (x,t)
            operator()( t, x, y); // calls the above operator
            dg::blas1::axpby( 1., x, -alpha, y);
        };
        dg::blas1::copy( rhs, omega); // use rhs as initial guess
        unsigned number = m_pcg.solve( wrapper, omega, rhs, 1., m_weights, m_eps_time);
    }
    private:
    Explicit<Geometry,Matrix,Container>& m_exp;
    dg::PCG< Container> m_pcg;
    Container m_weights;
    value_type m_eps_time;
};

// Construct equations
Explicit<...> ex( ...);
Implicit<...> im(ex, ...);

// The timestepper
dg::Adaptive< dg::ARKStep< dg::x::DVec>> adapt(tableau, omega);

// The timeloop
dg::AdaptiveTimeloop<dg::x::DVec> timeloop( adapt, std::tie( ex, im, im), 
                    dg::pid_control, dg::l2norm, rtol, atol);
```


## Implicit mass-matrix advection-diffusion equation
Example equation
\begin{align}
    M(y)\frac{d y}{d t} &= F(y,t)
\end{align}
We intend to solve the resulting implicit equation with multigrid nested iteration.
```cpp
template<...>
struct MassMatrix
{
    MassMatrix(...){}
    void operator()( double a, double t, const Container& y, const Container& k, 
                    double b, Container& result)
    {
        // Compute result = a M(y,t) k + b result
        //...
    }
};
template<...>
struct F
{
    F(...){}
    void operator()( double t, const Container& y,  Container& k)
    {
        // Compute k = F(y,t)
    }
};
template< ...>
class Implicit
{
    Implicit( ...){
        //construct nested objects
    }
    void operator()( double t, const Container& y,  Container& k)
    {
        m_F[0](t,y,m_tmp);
        std::vector<std::function<void( const Container&, Container&)> solvers;
        for( unsigned u=0; u<m_nested.stages(); u++)
        // We need to solve M(y,t)k = F(y,t) 
        // construct lambdas (see Solver documentation on how to do it)
        nested_iterations( solvers, k, m_tmp, ...);
    }
    void operator()( double alpha, double t, Container& y, const Container& rhs)
    {
        
        std::vector<std::function<void( const Container&, Container&)> solvers;
        for( unsigned u=0; u<m_nested.stages(); u++)
        // We need to solve M(y,t)(y-y*) - alpha F(y,t) = 0
        // construct lambdas (see Solver documentation on how to do it)
        nested_iterations( solvers, y, 0., ...);
    }
    private:
    dg::Nested< ...> m_nested;
    std::vector<MassMatrix> m_mass;
    std::vector<F> m_F;
    std::vector< dg::PCG> m_linear_solvers;
    std::vector< dg::AndersonAcc> m_nonlinear_solvers;
};

// in main
Implicit<...> imp(...);
dg::ImplicitMultistep<Vector> multistep("BDF-3-3", y0);
multistep.init( imp, t0, y0, dt);
multistep.step( imp, t0, y0);
```
